In [5]:
import numpy as np

# Note Features
S0 = 41.76                 # Initial stock price ($)
K = 25.06                  # Coupon barrier and downside threshold ($)
coupon_rate = 0.1025       # Contingent coupon rate per annum
coupon_payment = 25.625    # Contingent coupon per observation date ($)
principal = 1000           # Principal amount per note ($)
share_delivery_amount = 23.9464  # Shares delivered if below threshold at maturity
call_start_step = 2        # Issuer can call starting from the second observation date

# Market Parameters
r = 0.04                   # Risk-free interest rate (2%)
q = 0.02977                   # Dividend yield (3%)
sigma = 0.25               # Stock volatility (25%)

# Time Parameters
T = 2.0                    # Total time to maturity (2 years)
N = 1000                      # Number of time steps (quarterly observations)
dt = T / N                 # Time step size

# Binomial Tree Parameters
u = np.exp(sigma * np.sqrt(dt))        # Up factor
d = np.exp(-sigma * np.sqrt(dt))       # Down factor
p = (np.exp((r - q) * dt) - d) / (u - d)  # Risk-neutral probability

# Check for arbitrage
if not (0 < p < 1):
    raise ValueError("Risk-neutral probability is not between 0 and 1. Check parameters.")

# Build Stock Price Tree
stock_tree = np.zeros((N + 1, N + 1))
for i in range(N + 1):
    for j in range(i + 1):
        stock_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)

# Initialize Payoff Matrix
value_tree = np.zeros_like(stock_tree)

# Determine Payoffs at Maturity
for j in range(N + 1):
    S_T = stock_tree[j, N]
    if S_T >= K:
        # Receive principal back
        value_tree[j, N] = principal + (coupon_payment if S_T >= K else 0)
    else:
        # Receive share delivery amount
        value_tree[j, N] = share_delivery_amount * S_T

# Backward Induction
for i in range(N - 1, -1, -1):
    for j in range(i + 1):
        S_t = stock_tree[j, i]
        # Calculate expected continuation value
        continuation_value = np.exp(-r * dt) * (p * value_tree[j, i + 1] + (1 - p) * value_tree[j + 1, i + 1])
        
        # Add contingent coupon if barrier is breached
        if S_t >= K:
            continuation_value += coupon_payment
        
        # Check if issuer will call the note
        if i >= call_start_step:
            # Calculate call payoff
            call_payoff = principal + (coupon_payment if S_t >= K else 0)
            # Issuer will call if it's optimal
            node_value = min(call_payoff, continuation_value)
        else:
            node_value = continuation_value
        
        value_tree[j, i] = node_value

# Output the Note's Present Value
note_value = value_tree[0, 0]
print(f"The estimated fair value of the note is: ${note_value:.2f}")

The estimated fair value of the note is: $1076.71


In [3]:
value_tree

array([[1049.3572047 , 1041.04486074, 1025.625     , 1025.625     ,
        1025.625     , 1025.625     , 1025.625     , 1025.625     ,
        1025.625     ],
       [   0.        , 1027.56890549, 1025.625     , 1025.625     ,
        1025.625     , 1025.625     , 1025.625     , 1025.625     ,
        1025.625     ],
       [   0.        ,    0.        ,  999.51066174, 1025.625     ,
        1025.625     , 1025.625     , 1025.625     , 1025.625     ,
        1025.625     ],
       [   0.        ,    0.        ,    0.        ,  945.13808075,
        1012.47309939, 1025.625     , 1025.625     , 1025.625     ,
        1025.625     ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
         851.85321836,  970.25730051, 1025.625     , 1025.625     ,
        1025.625     ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,  709.85939171,  888.44954504, 1025.625     ,
        1025.625     ],
       [   0.        ,    0.        ,   